In [1]:
from pyresparser import ResumeParser
import pandas as pd
from operator import itemgetter

In [2]:
import os
os.chdir('C:\\Users\\n10301135\\OneDrive - Queensland University of Technology\\Datathon')

In [3]:
resume_path = 'scripts/Gaurav Chandra Resume.pdf'
job_file_path = 'scripts/Seek DS Detailed.csv'


job_data = pd.read_csv(job_file_path)
def extract_resume_data():
    match_description = []
    data = ResumeParser(resume_path).get_extracted_data()
    for index, row in job_data.iterrows():
        new_row = row.drop(labels=['Title', 'Company', 'Location', 'Date', 'Job Type', 'Description'])
        job_skills = []
        for column in new_row:
            if isinstance(column, str):
                job_skills.append(column)
        if len(job_skills) > 0:
            matched = set(data.get('skills')) & set(job_skills)
            unmatched = set(job_skills) - set(data.get('skills'))
            if len(matched) > 0:
                match_description.append(
                    [row['Title'], list(matched), list(unmatched), len(matched), (len(matched) / len(job_skills) * 100)])
    match_description = sorted(match_description, key=itemgetter(4), reverse=True)
    return pd.DataFrame(match_description, columns=['jobtitle', 'matched_skills', 'unmatched_skills', 'num_skills', 'matched_precentage'])

In [4]:
result = extract_resume_data()

C:\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
#title, matched, unmatched, len_matched, percent
result.head(10)

,jobtitle,matched_skills,unmatched_skills,num_skills,matched_precentage
0,Senior Data Engineer,[Architecture],[Enterprise],1,50.000000
1,Consultant Technical Trainer (Analytics),"[Java, Python]","[C++, FinTech]",2,50.000000
2,Lead Quantitative Analyst / Data Scientist - B...,"[Python, R]","[Processing, Government, ASAP]",2,40.000000
3,Senior Data Scientist | Contract | Iconic orga...,"[Python, R]","[Finance, Alteryx, Keras, Matplotlib]",2,33.333333
4,"Manager , Credit Risk (SME) - Data Science",[Architecture],"[Digital, IT]",1,33.333333
5,Data Engineer - AWS/Spark/Startup,[R],"[Learning, Cloud]",1,33.333333
6,Data Engineer,"[Java, Database, Python]","[Kimball, Catalogue, Azure, Snowflake, Word, S...",3,30.000000
7,Data Scientist - Adelaide or Canberra,[Python],"[AWS, Processing, Advanced]",1,25.000000
8,Data Scientist | Contract | Iconic organisation |,[Python],"[Cloudera, Microsoft, Azure]",1,25.000000
9,Lead Data Analyst,"[Operations, Python]","[Intelligence, Road, EDW, Azure, Analytics, SQL]",2,25.000000


In [6]:
import matplotlib.pyplot as plt
import numpy as np

In [8]:
from collections import Counter

In [19]:
len(result)

57

In [12]:
result['jobtitle'].head(10)

0                                 Senior Data Engineer
1             Consultant Technical Trainer (Analytics)
2    Lead Quantitative Analyst / Data Scientist - B...
3    Senior Data Scientist | Contract | Iconic orga...
4           Manager , Credit Risk (SME) - Data Science
5                    Data Engineer - AWS/Spark/Startup
6                                        Data Engineer
7                Data Scientist - Adelaide or Canberra
8    Data Scientist | Contract | Iconic organisation |
9                                    Lead Data Analyst
Name: jobtitle, dtype: object

In [26]:
result.to_csv('resumeripper_result.csv')

In [13]:
unmatched_skills = []
for i in result['unmatched_skills'].values:
    unmatched_skills += i

In [24]:
topunmatchedskills = []
for key, value in sorted(Counter(unmatched_skills).items(), key=lambda item: item[1], reverse=True)[:n]:
    topunmatchedskills.append([key, int(value*100/57)])
topunmatchedskills_df = pd.DataFrame(topunmatchedskills, columns=['skill', 'percent_alljobs'])

In [25]:
topunmatchedskills_df.head()

,skill,percent_alljobs
0,SQL,33
1,AWS,28
2,Spark,22
3,Cloud,21
4,Learning,19


In [27]:
topunmatchedskills_df.to_csv('unmatchedskills.csv')